In [47]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [48]:
views_df = duckdb.sql("SELECT * FROM 'data/content_views.parquet'").df()
content_df = duckdb.sql("SELECT * FROM 'data/content_metadata.parquet'").df()
adventurers_df = duckdb.sql("SELECT * FROM 'data/adventurer_metadata.parquet'").df()

In [49]:
df = views_df.merge(content_df, on='content_id').merge(adventurers_df, on='adventurer_id')

In [50]:
content_features = ["genre_id", "language_code", "minutes"]
adventurer_features = ["honorific", "gender", "age", "region"]

all_features = content_features + adventurer_features

categorical = ["genre_id", "language_code", "honorific", "gender", "region"]
numerical = ["minutes", "age"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
    ("num", StandardScaler(), numerical)
])

X_all = preprocessor.fit_transform(df[all_features])
df_features = pd.DataFrame.sparse.from_spmatrix(X_all, index=df.index)


In [51]:
adventurer_profiles = df_features.groupby(df["adventurer_id"]).mean()

In [52]:
print(adventurer_profiles)

                    0         1      2         3         4         5    \
adventurer_id                                                            
113g                  0       0.2      0       0.2         0         0   
113y                  0         0      0         0         0         0   
114t                  0       1.0      0         0         0         0   
114z                  0         0      0         0         0       1.0   
115m                  0         0      0         0       0.5       0.5   
...                 ...       ...    ...       ...       ...       ...   
zzip                  0  0.285714      0         0         0         0   
zzoq           0.083333  0.041667  0.125  0.041667  0.208333  0.291667   
zzrt                  0      0.25      0         0      0.25       0.5   
zzux               0.25         0      0      0.25         0         0   
zzz8                0.4         0      0         0       0.2       0.2   

                    6         7     8

In [32]:
content_X = preprocessor.transform(
    content_df[content_features].assign(
        honorific="", gender="", age=0, region=""  # placeholders for missing cols
    )
)

content_features_df = pd.DataFrame.sparse.from_spmatrix(
    content_X, index=content_df["content_id"]
)


In [57]:
knn = NearestNeighbors(metric="cosine", algorithm="auto")
knn.fit(content_features_df)


NearestNeighbors(metric='cosine')

In [58]:
def recommend_for_adventurer(adventurer_id, n_recs=5):
    if adventurer_id not in adventurer_profiles.index:
        return pd.DataFrame()
    
    profile_vec = adventurer_profiles.loc[[adventurer_id]].to_numpy()
    distances, indices = knn.kneighbors(profile_vec, n_neighbors=n_recs*2)  # get extra
    
    candidate_ids = content_features_df.iloc[indices[0]].index
    
    # Filter out already watched
    watched = set(df.loc[df["adventurer_id"] == adventurer_id, "content_id"])
    new_recs = [cid for cid in candidate_ids if cid not in watched]
    
    # Limit to top n_recs
    recs = content_df[content_df["content_id"].isin(new_recs[:n_recs])]
    return recs[["content_id", "title", "genre_id", "language_code", "minutes"]]


In [59]:
print(recommend_for_adventurer("xir8", n_recs=5))


    content_id                                            title genre_id  \
8         7g3r                 Gelatina Grove: Cryptic Secrets!      COM   
207       9c24               The Holiday Reflections of Balance      HOL   
305       s5ig  Understanding the Holiday: Majestic Discoveries      HOL   
455       4mky              Holiday Tranquil Strength: A Legend      HOL   
587       zgv9                  The Holiday Methods of Ancestry      HOL   

    language_code  minutes  
8              RP     4.94  
207            RP     4.90  
305            RP     5.25  
455            RP     4.83  
587            RP     5.39  


In [60]:
q = f"""
SELECT * 
FROM 'data/content_views.parquet'
LEFT JOIN 'data/adventurer_metadata.parquet' USING(adventurer_id)
LEFT JOIN 'data/content_metadata.parquet' USING(content_id)
WHERE adventurer_id = 'xir8'
"""

print(duckdb.sql(q))

┌────────────┬────────────────┬────────┬───────────────┬──────────────┬─────────────┬───────────┬───────────┬──────────────┬───────┬────────────┬────────────┬─────────┬───────┬──────────────────┬────────────────┬──────────────┬─────────┬────────────────────────────────────────────────┬──────────┬─────────┬───────────────┬───────────┬──────────┬──────────────┬───────┐
│ content_id │ seconds_viewed │ rating │ adventurer_id │ publisher_id │ playlist_id │   month   │    day    │ day_of_month │ year  │    name    │ honorific  │ gender  │  age  │ primary_language │ favorite_genre │    region    │ studio  │                     title                      │ genre_id │ minutes │ language_code │   month   │   day    │ day_of_month │ year  │
│  varchar   │     double     │ double │    varchar    │   varchar    │   varchar   │  varchar  │  varchar  │    int64     │ int64 │  varchar   │  varchar   │ varchar │ int64 │     varchar      │    varchar     │   varchar    │ varchar │                    var